In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from scipy.spatial import distance
import speech_recognition as sr
import threading

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Blink Detection Parameters
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3
blink_counter = 0
blink_total = 0
cheat_intensity = 0
voice_detected = False  # To track voice detection

# Iris and Eye Landmarks
LEFT_EYE = [362, 385, 387, 263, 373, 380]  # Correct landmarks for EAR
RIGHT_EYE = [33, 160, 158, 133, 153, 144]  # Correct landmarks for EAR
LEFT_IRIS = [474, 475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]

# Lip Landmarks
UPPER_LIP_INDEX = 13
LOWER_LIP_INDEX = 14
lip_open_count = 0
lip_open = False

# EAR Calculation Function
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Extract Eye Landmarks from Face Mesh
def extract_eye_landmarks(landmarks, indices):
    return np.array([(landmarks[i].x, landmarks[i].y) for i in indices])

# Voice Detection Function
def detect_voice():
    global voice_detected
    recognizer = sr.Recognizer()
    mic = sr.Microphone()

    with mic as source:
        recognizer.adjust_for_ambient_noise(source)
        while True:
            try:
                audio = recognizer.listen(source, timeout=2, phrase_time_limit=2)
                recognizer.recognize_google(audio)  # Just to ensure there's some sound
                voice_detected = True
            except sr.WaitTimeoutError:
                voice_detected = False
            except sr.UnknownValueError:
                voice_detected = False

# Gaze Direction Utility Functions
def get_landmark_coordinates(landmarks, index, image_shape):
    x = int(landmarks[index].x * image_shape[1])
    y = int(landmarks[index].y * image_shape[0])
    return (x, y)

def get_gaze_direction(landmarks, image_shape):
    left_eye = get_landmark_coordinates(landmarks, 33, image_shape)
    right_eye = get_landmark_coordinates(landmarks, 263, image_shape)
    nose_tip = get_landmark_coordinates(landmarks, 1, image_shape)
    
    eye_center = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)
    nose_to_eye_vector = np.array([eye_center[0] - nose_tip[0], eye_center[1] - nose_tip[1]])
    
    if nose_to_eye_vector[0] > 20:
        return "head turned Left"
    elif nose_to_eye_vector[0] < -20:
        return "head turned Right"
    else:
        return "Looking Forward"

# Start voice detection in a separate thread
voice_thread = threading.Thread(target=detect_voice, daemon=True)
voice_thread.start()

# Start webcam feed
cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        img_h, img_w = image.shape[:2]

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())

                # Extract eye landmarks
                left_eye = extract_eye_landmarks(face_landmarks.landmark, LEFT_EYE)
                right_eye = extract_eye_landmarks(face_landmarks.landmark, RIGHT_EYE)

                # Calculate EAR for both eyes
                left_eye_ear = eye_aspect_ratio(left_eye)
                right_eye_ear = eye_aspect_ratio(right_eye)
                ear = (left_eye_ear + right_eye_ear) / 2.0

                # Detect blink based on EAR threshold
                if ear < EYE_AR_THRESH:
                    blink_counter += 1
                else:
                    if blink_counter >= EYE_AR_CONSEC_FRAMES:
                        blink_total += 1
                        cheat_intensity += 1
                    blink_counter = 0

                cv2.putText(image, f"Blinks: {blink_total}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                # Detect gaze direction using nose and eye center
                gaze_direction = get_gaze_direction(face_landmarks.landmark, image.shape)
                cv2.putText(image, f"Gaze: {gaze_direction}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

                # Detect lip movement
                upper_lip = face_landmarks.landmark[UPPER_LIP_INDEX]
                lower_lip = face_landmarks.landmark[LOWER_LIP_INDEX]
                lip_distance = abs(upper_lip.y - lower_lip.y)

                if lip_distance > 0.05:  # Adjust threshold if needed
                    if not lip_open:
                        lip_open_count += 1
                        lip_open = True
                else:
                    lip_open = False

                cv2.putText(image, f"Lip Opens: {lip_open_count}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

                # Calculate and display iris positions
                mesh_points = np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) for p in face_landmarks.landmark])

                (l_cx, l_cy), l_radius = cv2.minEnclosingCircle(mesh_points[LEFT_IRIS])
                center_left = np.array([l_cx, l_cy], dtype=np.int32)
                cv2.circle(image, center_left, int(l_radius), (0, 255, 0), 2, cv2.LINE_AA)

                (r_cx, r_cy), r_radius = cv2.minEnclosingCircle(mesh_points[RIGHT_IRIS])
                center_right = np.array([r_cx, r_cy], dtype=np.int32)
                cv2.circle(image, center_right, int(r_radius), (0, 255, 0), 2, cv2.LINE_AA)

                # Get horizontal position of the iris relative to the eye width
                left_eye_width = np.linalg.norm(left_eye[0] - left_eye[3])
                right_eye_width = np.linalg.norm(right_eye[0] - right_eye[3])

                left_gaze_ratio = (center_left[0] - left_eye[0][0]) / left_eye_width
                right_gaze_ratio = (center_right[0] - right_eye[0][0]) / right_eye_width

                # Determine the direction of gaze using iris positions
                iris_gaze_direction = "Center"
                if left_gaze_ratio < 0.5 and right_gaze_ratio < 0.5:
                    iris_gaze_direction = "Looking Left"
                elif left_gaze_ratio > 0.5 and right_gaze_ratio > 0.5:
                    iris_gaze_direction = "Looking Right"

                cv2.putText(image, f'Iris Gaze: {iris_gaze_direction}', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Display voice detection status
        if voice_detected:
            cv2.putText(image, "Voice Detected!", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Display the image
        cv2.imshow('Gaze, Blink, Voice, Lip, and Iris Detector', image)

        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()


I0000 00:00:1729866614.685490 59467917 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1729866614.701567 59468843 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1729866614.709347 59468843 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/prathamhandique/Downloads/anaconda3/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
